In [1]:
#encoding utf-8
import numpy as np
import pandas as pd
from IPython.display import Image
import json
import urllib2
from time import sleep

In [2]:
DTFQA = pd.read_pickle('./db/DTFQA_v02')
DTFQA.iloc[0:5,:]

,No.,utterances,intent,entity,nlp_class,nlp_name
0,1,請問洗手間在哪?,問門市,"分店::內部資訊,洗手間;",NLP_Major_05,店內資訊
1,1,你們化妝室怎麼走?,問門市,"分店::內部資訊,化妝室;",NLP_Major_05,店內資訊
2,1,我想上廁所,問門市,"分店::內部資訊,廁所;",NLP_Major_05,店內資訊
3,3,什麼時候不需要排隊?,問門市,"分店::候位,排隊;",NLP_Major_05,店內資訊
4,3,哪時候人最少,問門市,"分店::候位,人最少;",NLP_Major_05,店內資訊


In [3]:
##################################
#     Jieba class definition
##################################
#data={u'sentence': u'我想吃晚餐'}

class jieba_seg(object):
    def __init__(self):
        self.jebareq = urllib2.Request('http://104.199.137.211/dtf_poc')
        #req = urllib2.Request('http://104.199.137.211/jieba_segmentation')
        self.ret_json = ()
        proxy = urllib2.ProxyHandler({})
        opener = urllib2.build_opener(proxy)
        urllib2.install_opener(opener)      
        
    def do_seg(self,data):
        ToDMDataJson=json.dumps(data)
         
        self.jebareq.add_header('Content-Type', 'application/json')
        f = urllib2.urlopen(self.jebareq, ToDMDataJson, timeout=12)                    
        decodejson= json.load(f)
        self.ret_json = json.dumps(decodejson, ensure_ascii=False)
        return self


In [26]:
# single word segmentation test
term = np.array([])
pos  = np.array([])
data={u'sentence': u'你們的茶很好喝?'}
jeba = jieba_seg()   
jeba.do_seg(data)
j_format = json.loads(jeba.ret_json)
#print repr(j_format['term']).decode("unicode-escape")
print jeba.ret_json

{"term": ["你們", "的", "茶", "很", "好喝", "?"], "pos": ["nh", "uj", "n", "d", "praise", "x"]}


In [5]:
# full db sentence do word segmentation
term = np.array([])
pos  = np.array([])

DTFQA['term'] = 'small'
DTFQA['pos'] = 'small'
for i in range(len(DTFQA['No.'])):
    print DTFQA.get_value(i,'utterances')
    data = {u'sentence': DTFQA.get_value(i,'utterances')}
    jeba = jieba_seg()   
    jeba.do_seg(data)
    j_format = json.loads(jeba.ret_json)
    #pos = np.vstack((pos,j_format['pos']))
    #pos = np.concatenate((pos,np.array([j_format['pos']])))
    DTFQA['term'][i] = j_format['term']
    DTFQA['pos'][i] = j_format['pos']
    print repr(j_format['term']).decode("unicode-escape")
    print jeba.ret_json
    sleep(0.5)

請問洗手間在哪?
[u'請問', u'洗手間', u'在哪', u'?']
{"term": ["請問", "洗手間", "在哪", "?"], "pos": ["qhead", "branchinnerinfob", "where", "x"]}
你們化妝室怎麼走?
[u'你們', u'化妝室', u'怎麼走', u'?']
{"term": ["你們", "化妝室", "怎麼走", "?"], "pos": ["nh", "branchinnerinfob", "where", "x"]}
我想上廁所
[u'我', u'想', u'上', u'廁所']
{"term": ["我", "想", "上", "廁所"], "pos": ["r", "v", "f", "branchinnerinfob"]}
什麼時候不需要排隊?
[u'什麼時候', u'不需要', u'排隊', u'?']
{"term": ["什麼時候", "不需要", "排隊", "?"], "pos": ["when", "ineg", "branchinnerinfoa", "x"]}
哪時候人最少
[u'哪時候', u'人最少']
{"term": ["哪時候", "人最少"], "pos": ["when", "branchinnerinfoa"]}
哪個時段不用候位
[u'哪個時段', u'不用', u'候位']
{"term": ["哪個時段", "不用", "候位"], "pos": ["when", "ineg", "branchinnerinfoa"]}
你們員工的待遇是不是很高?
[u'你們', u'員工', u'的', u'待遇', u'是不是', u'很高', u'?']
{"term": ["你們", "員工", "的", "待遇", "是不是", "很高", "?"], "pos": ["nh", "waiter", "uj", "waiterinfo", "qhead", "more", "x"]}
服務人員的薪水很高吧
[u'服務人員', u'的', u'薪水', u'很高', u'吧']
{"term": ["服務人員", "的", "薪水", "很高", "吧"], "pos": ["waiter", "uj", "waiterinfo", "more", "y

In [5]:
######### output
DTFQA.to_pickle('./db/DTFQA_v03')

In [6]:
######### verify output data
DTFQA = pd.read_pickle('./db/DTFQA_v03')
DTFQA.iloc[0:5,:]

,No.,utterances,intent,entity,nlp_class,nlp_name,term,pos
0,1,請問洗手間在哪?,問門市,"分店::內部資訊,洗手間;",NLP_Major_05,店內資訊,"[請問, 洗手間, 在哪, ?]","[qhead, branchinnerinfob, where, x]"
1,1,你們化妝室怎麼走?,問門市,"分店::內部資訊,化妝室;",NLP_Major_05,店內資訊,"[你們, 化妝室, 怎麼走, ?]","[nh, branchinnerinfob, where, x]"
2,1,我想上廁所,問門市,"分店::內部資訊,廁所;",NLP_Major_05,店內資訊,"[我, 想, 上, 廁所]","[r, v, f, branchinnerinfob]"
3,3,什麼時候不需要排隊?,問門市,"分店::候位,排隊;",NLP_Major_05,店內資訊,"[什麼時候, 不需要, 排隊, ?]","[when, ineg, branchinnerinfoa, x]"
4,3,哪時候人最少,問門市,"分店::候位,人最少;",NLP_Major_05,店內資訊,"[哪時候, 人最少]","[when, branchinnerinfoa]"


In [ ]:
###########################################################################
#######################  以下都是測試程式 ##################################

In [ ]:
print jeba.ret_json

In [ ]:
j_format = json.loads(jeba.ret_json)
print repr(j_format['term']).decode("unicode-escape")

In [ ]:
DTFQA